In [1]:
class Person:
    def hello(arg='default'):
        print(f'Hello, from args={arg}')

In [2]:
# Normal Function ---
Person.hello(100)

# Method(bound to instance)
p = Person()
p.hello()

Hello, from args=100
Hello, from args=<__main__.Person object at 0x051470B0>


But can we create a function which is bound to a class, which do not interact with instances, but only interacts with class no matter how it is being called ?

**Yes**, we can with the help of ```@classmethod```

## Classmethod

In [3]:
class MyClass:
    def hello():
        print('Hello...')
    
    def instance_hello(arg):
        print(f'hello from {arg}')
    
    # this will make a function bound to a class not to an instance.
    @classmethod
    def class_hello(arg):
        print(f'hello from {arg}')

In [4]:
a = MyClass()
# instance bounded, Noraml function
a.instance_hello, MyClass.instance_hello

(<bound method MyClass.instance_hello of <__main__.MyClass object at 0x05147110>>,
 <function __main__.MyClass.instance_hello(arg)>)

In [5]:
# In both of the cases you will see that class_hello function is class-bounded function
a.class_hello, MyClass.class_hello

(<bound method MyClass.class_hello of <class '__main__.MyClass'>>,
 <bound method MyClass.class_hello of <class '__main__.MyClass'>>)

In [6]:
from datetime import datetime, timezone, timedelta

In [7]:
class Timer:
    tz = timezone.utc
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset))
        

In [8]:
Timer.set_tz(-7, 'MST')

In [9]:
Timer.tz

datetime.timezone(datetime.timedelta(-1, 61200))

In [10]:
t1 = Timer()
t2 = Timer()

t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(-1, 61200)),
 datetime.timezone(datetime.timedelta(-1, 61200)))

In [11]:
Timer.set_tz(-8, 'PST')
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(-1, 57600)),
 datetime.timezone(datetime.timedelta(-1, 57600)))

In [12]:
class Timer:
    tz = timezone.utc
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset))
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)

In [13]:
t = Timer()
# Static method, it will give same output as static method is just a normal
# which can be defined even outside class as it is not bound to anyone.

t.current_dt_utc(), Timer.current_dt_utc()

(datetime.datetime(2020, 8, 4, 7, 14, 14, 669359, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 8, 4, 7, 14, 14, 669359, tzinfo=datetime.timezone.utc))

In [14]:
class Timer:
    tz = timezone.utc
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset))
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)

In [15]:
Timer.current_dt_utc(), Timer.current_dt()

(datetime.datetime(2020, 8, 4, 7, 18, 52, 215968, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 8, 4, 7, 18, 52, 215968, tzinfo=datetime.timezone.utc))

In [16]:
t1 = Timer()
t2 = Timer()

t1.current_dt_utc(), t2.current_dt()

(datetime.datetime(2020, 8, 4, 7, 20, 18, 265935, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 8, 4, 7, 20, 18, 265935, tzinfo=datetime.timezone.utc))

In [17]:
# We can change tz for instance--
# t1.tz = 'some value'

# Change tz for class --

t1.set_tz(-7, 'MST')
# or
# Timer.set_tz(-7, 'MST')


In [20]:
# You can see that timezone has been changed and same for both instances.

t1.current_dt(), t2.current_dt()
# t1.tz, t2.tz

(datetime.datetime(2020, 8, 4, 0, 26, 5, 519885, tzinfo=datetime.timezone(datetime.timedelta(-1, 61200))),
 datetime.datetime(2020, 8, 4, 0, 26, 5, 519885, tzinfo=datetime.timezone(datetime.timedelta(-1, 61200))))

In [21]:
class TimerError(Exception):
    '''A custom exception user for Timer Class'''

In [33]:
class Timer:
    tz = timezone.utc
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset))
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)
    
    def start(self):
        self._time_start = self.current_dt_utc()
        self._time_end = None
    
    def stop(self):
        if self._time_start is None:
            raise TimeError('Timer Must be started before it can be stopped')
        self._time_end = self.current_dt_utc()
    
    @property
    def start_time(self):
        if self._time_start is None:
            raise TimeError('Timer has not been started')
        return self._time_start.astimezone(self.tz)
    
    @property
    def end_time(self):
        if self._time_end is None:
            raise TimerError('Timer has not been stopped')
        return self._time_end.astimezone(self.tz)
    
    @property
    def elapsed(self):
        if self._time_start is None:
            raise TimerError('Timer must be started before an elapsed time can be calculated')
        if self._time_end is None:
            elapsed_time = self.current_dt_utc() - self._time_start
        else:
            elapsed_time = self._time_end - self._time_start
        return elapsed_time.total_seconds()

In [34]:
from time import sleep

t1 = Timer()
t1.start()
sleep(2)
t1.stop()

print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2020-08-04 07:52:32.329460+00:00
End time: 2020-08-04 07:52:34.329657+00:00
Elapsed: 2.000197 seconds


In [35]:
t2 = Timer()
t2.start()
sleep(3)
t2.stop()
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2020-08-04 07:52:40.658756+00:00
End time: 2020-08-04 07:52:43.659335+00:00
Elapsed: 3.000579 seconds


In [36]:
# Lets change the time zone now
Timer.set_tz(-7, 'MST')

In [37]:
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2020-08-04 00:52:32.329460-07:00
End time: 2020-08-04 00:52:34.329657-07:00
Elapsed: 2.000197 seconds
